In [63]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import joblib

# Load the dataset from CSV files
ambulance_file_path = 'ambulance.csv'
traffic_file_path = 'traffic.csv'

ambulance_data = pd.read_csv(ambulance_file_path)
traffic_data = pd.read_csv(traffic_file_path)

# Combine the datasets
dataset = pd.concat([ambulance_data, traffic_data], ignore_index=True)

# Split the dataset into features and labels
features = dataset.iloc[:, :-1].values
labels = dataset.iloc[:, -1].values

# Convert labels to numerical values
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    features, labels_encoded, test_size=0.27, random_state=48
)

# Define and train the classifier
classifier = make_pipeline(StandardScaler(), SVC())
classifier.fit(X_train, y_train)

# Evaluate the classifier
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

# Save the classifier model and label encoder
classifier_model_path = 'classifier_model.pkl'
label_encoder_path = 'label_encoder.pkl'

joblib.dump(classifier, classifier_model_path)
joblib.dump(label_encoder, label_encoder_path)

print("Classifier model and label encoder saved successfully.")


Accuracy: 0.9814814814814815
Precision: 0.9820987654320987
Recall: 0.9814814814814815
F1 Score: 0.9814494536044173
Classifier model and label encoder saved successfully.


In [65]:
import librosa
import numpy as np
from sklearn.preprocessing import StandardScaler
import joblib

# Load the trained classifier and label encoder
classifier = joblib.load('classifier_model.pkl')  # Replace with the path to your trained classifier model
label_encoder = joblib.load('label_encoder.pkl')  # Replace with the path to your label encoder

# Define a function to extract features from the WAV file
def extract_features(file_path):
    audio, sr = librosa.load(file_path, sr=None)  # Load the audio file

    # Extract features
    chroma_stft = librosa.feature.chroma_stft(audio, sr=sr)
    spectral_centroid = librosa.feature.spectral_centroid(audio, sr=sr)[0]
    spectral_bandwidth = librosa.feature.spectral_bandwidth(audio, sr=sr)[0]
    spectral_rolloff = librosa.feature.spectral_rolloff(audio, sr=sr)[0]
    rmse = librosa.feature.rms(audio)[0]
    zero_crossing_rate = librosa.feature.zero_crossing_rate(audio)[0]

    # Concatenate features into a single feature vector
    features = np.hstack(
        (chroma_stft, spectral_centroid.reshape(-1, 1), spectral_bandwidth.reshape(-1, 1),
         spectral_rolloff.reshape(-1, 1), rmse.reshape(-1, 1), zero_crossing_rate.reshape(-1, 1))
    )
    return features

# Define a function to classify a given WAV file
def classify_audio(file_path):
    features = extract_features(file_path)  # Extract features from the WAV file

    # Scale the features using the same scaler used during training
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(features)

    # Classify the audio sample
    predicted_label = label_encoder.inverse_transform(classifier.predict(scaled_features))

    return predicted_label[0]  # Return the predicted label as a string

# Use the classify_audio function to classify a given WAV file
wav_file_path = 'sound_1.wav'  # Replace with the path to your input WAV file

predicted_label = classify_audio(wav_file_path)
print("Predicted Label:", predicted_label)


TypeError: ignored

In [68]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import librosa
from scipy.io import wavfile
import joblib

# Load the datasets
ambulance_data = pd.read_csv('ambulance.csv')
traffic_data = pd.read_csv('traffic.csv')

# Combine the datasets
combined_data = pd.concat([ambulance_data, traffic_data], ignore_index=True)

# Separate features and labels
features = combined_data.iloc[:, :-1]
labels = combined_data.iloc[:, -1]

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, encoded_labels, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train the classifier
classifier = SVC()
classifier.fit(X_train_scaled, y_train)

# Predict on the test set
y_pred = classifier.predict(X_test_scaled)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

# Save the classifier model and label encoder
joblib.dump(classifier, 'classifier_model.pkl')
joblib.dump(label_encoder, 'label_encoder.pkl')

# Define a function to classify a single WAV file
def classify_audio(file_path):
    audio, sr = librosa.load(file_path, sr=None)  # Load the audio file

    # Extract features
    chroma_stft = librosa.feature.chroma_stft(y=audio, sr=sr)
    spectral_centroid = librosa.feature.spectral_centroid(y=audio, sr=sr)[0]
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=audio, sr=sr)[0]
    spectral_rolloff = librosa.feature.spectral_rolloff(y=audio, sr=sr)[0]
    rmse = librosa.feature.rms(y=audio)[0]
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y=audio)[0]

    # Concatenate features into a single feature vector
    features = np.hstack(
        (chroma_stft.T, spectral_centroid.reshape(-1, 1), spectral_bandwidth.reshape(-1, 1),
         spectral_rolloff.reshape(-1, 1), rmse.reshape(-1, 1), zero_crossing_rate.reshape(-1, 1))
    )

    # Scale the features
    scaled_features = scaler.transform(features)

    # Classify the audio sample
    predicted_label = label_encoder.inverse_transform(classifier.predict(scaled_features))

    return predicted_label[0]  # Return the predicted label as a string

# Use the classify_audio function to classify a single WAV file
wav_file_path = 'sound_1.wav'  # Replace with the path to your input WAV file

predicted_label = classify_audio(wav_file_path)
print("Predicted Label:", predicted_label)


Accuracy: 0.975
Precision: 0.9722222222222222
Recall: 0.9722222222222222
F1 Score: 0.9722222222222222


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


ValueError: ignored

In [70]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import librosa
from scipy.io import wavfile
import joblib

# Load the datasets
ambulance_data = pd.read_csv('ambulance.csv')
traffic_data = pd.read_csv('traffic.csv')

# Combine the datasets
combined_data = pd.concat([ambulance_data, traffic_data], ignore_index=True)

# Separate features and labels
features = combined_data.iloc[:, :-1]
labels = combined_data.iloc[:, -1]

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, encoded_labels, test_size=0.2, random_state=42)

# Select the relevant features for scaling
selected_features = ['chroma_stft', 'spectral_centroid', 'spectral_bandwidth', 'spectral_rolloff', 'rmse', 'zero_crossing_rate']
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

# Scale the selected features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_selected)
X_test_scaled = scaler.transform(X_test_selected)

# Train the classifier
classifier = SVC()
classifier.fit(X_train_scaled, y_train)

# Predict on the test set
y_pred = classifier.predict(X_test_scaled)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

# Save the classifier model and label encoder
joblib.dump(classifier, 'classifier_model.pkl')
joblib.dump(label_encoder, 'label_encoder.pkl')

# Define a function to classify a single WAV file
def classify_audio(file_path):
    audio, sr = librosa.load(file_path, sr=None)  # Load the audio file

    # Extract features
    chroma_stft = librosa.feature.chroma_stft(y=audio, sr=sr)
    spectral_centroid = librosa.feature.spectral_centroid(y=audio, sr=sr)[0]
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=audio, sr=sr)[0]
    spectral_rolloff = librosa.feature.spectral_rolloff(y=audio, sr=sr)[0]
    rmse = librosa.feature.rms(y=audio)[0]
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y=audio)[0]

    # Concatenate features into a single feature vector
    features = np.hstack(
        (chroma_stft.T, spectral_centroid.reshape(-1, 1), spectral_bandwidth.reshape(-1, 1),
         spectral_rolloff.reshape(-1, 1), rmse.reshape(-1, 1), zero_crossing_rate.reshape(-1, 1))
    )

    # Select the relevant features for scaling
    selected_features = features[:, :6]

    # Scale the selected features
    scaled_features = scaler.transform(selected_features)

    # Classify the audio sample
    predicted_label = label_encoder.inverse_transform(classifier.predict(scaled_features))

    return predicted_label[0]  # Return the predicted label as a string

# Use the classify_audio function to classify a single WAV file
wav_file_path = 'sound_403.wav'  # Replace with the path to your input WAV file

predicted_label = classify_audio(wav_file_path)
print("Predicted Label:", predicted_label)


Accuracy: 0.975
Precision: 0.9722222222222222
Recall: 0.9722222222222222
F1 Score: 0.9722222222222222
Predicted Label: ambulance


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [76]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import librosa
from scipy.io import wavfile
import joblib

# Load the datasets
ambulance_data = pd.read_csv('ambulance.csv')
traffic_data = pd.read_csv('traffic.csv')

# Combine the datasets
combined_data = pd.concat([ambulance_data, traffic_data], ignore_index=True)

# Separate features and labels
features = combined_data.iloc[:, :-1]
labels = combined_data.iloc[:, -1]

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, encoded_labels, test_size=0.2, random_state=25)

# Select the relevant features for scaling
selected_features = ['chroma_stft', 'spectral_centroid', 'spectral_bandwidth', 'spectral_rolloff', 'rmse', 'zero_crossing_rate']
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

# Scale the selected features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_selected)
X_test_scaled = scaler.transform(X_test_selected)

# Train the classifier
classifier = SVC()
classifier.fit(X_train_scaled, y_train)

# Predict on the test set
y_pred = classifier.predict(X_test_scaled)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

# Save the classifier model and label encoder
joblib.dump(classifier, 'classifier_model.pkl')
joblib.dump(label_encoder, 'label_encoder.pkl')

# Define a function to classify a single WAV file
def classify_audio(file_path):
    audio, sr = librosa.load(file_path, sr=None)  # Load the audio file

    # Extract features
    chroma_stft = librosa.feature.chroma_stft(y=audio, sr=sr)
    spectral_centroid = librosa.feature.spectral_centroid(y=audio, sr=sr)[0]
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=audio, sr=sr)[0]
    spectral_rolloff = librosa.feature.spectral_rolloff(y=audio, sr=sr)[0]
    rmse = librosa.feature.rms(y=audio)[0]
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y=audio)[0]

    # Concatenate features into a single feature vector
    features = np.hstack(
        (chroma_stft.T, spectral_centroid.reshape(-1, 1), spectral_bandwidth.reshape(-1, 1),
         spectral_rolloff.reshape(-1, 1), rmse.reshape(-1, 1), zero_crossing_rate.reshape(-1, 1))
    )

    # Select the relevant features for scaling
    selected_features = features[:, :6]

    # Scale the selected features
    scaled_features = scaler.transform(selected_features)

    # Classify the audio sample
    predicted_label = label_encoder.inverse_transform(classifier.predict(scaled_features))

    return predicted_label[1]  # Return the predicted label as a string

# Use the classify_audio function to classify a single WAV file
wav_file_path = 'sound_1.wav'  # Replace with the path to your input WAV file

predicted_label = classify_audio(wav_file_path)
print("Predicted Label:", predicted_label)


Accuracy: 0.9125
Precision: 0.9024390243902439
Recall: 0.925
F1 Score: 0.9135802469135802
Predicted Label: ambulance


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [77]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import librosa
from scipy.io import wavfile
import joblib

# Load the datasets
ambulance_data = pd.read_csv('ambulance.csv')
traffic_data = pd.read_csv('traffic.csv')

# Separate features and labels
ambulance_features = ambulance_data.iloc[:, :-1]
ambulance_labels = ambulance_data.iloc[:, -1]
traffic_features = traffic_data.iloc[:, :-1]
traffic_labels = traffic_data.iloc[:, -1]

# Encode labels separately for ambulance and traffic datasets
ambulance_label_encoder = LabelEncoder()
ambulance_encoded_labels = ambulance_label_encoder.fit_transform(ambulance_labels)
traffic_label_encoder = LabelEncoder()
traffic_encoded_labels = traffic_label_encoder.fit_transform(traffic_labels)

# Combine the encoded labels and features
combined_features = pd.concat([ambulance_features, traffic_features], ignore_index=True)
combined_encoded_labels = np.concatenate([ambulance_encoded_labels, traffic_encoded_labels])

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(combined_features, combined_encoded_labels, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train the classifier
classifier = SVC()
classifier.fit(X_train_scaled, y_train)

# Predict on the test set
y_pred = classifier.predict(X_test_scaled)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

# Save the classifier model and label encoders
joblib.dump(classifier, 'classifier_model.pkl')
joblib.dump(ambulance_label_encoder, 'ambulance_label_encoder.pkl')
joblib.dump(traffic_label_encoder, 'traffic_label_encoder.pkl')

# Define a function to classify a single WAV file
def classify_audio(file_path):
    audio, sr = librosa.load(file_path, sr=None)  # Load the audio file

    # Extract features
    chroma_stft = librosa.feature.chroma_stft(y=audio, sr=sr)
    spectral_centroid = librosa.feature.spectral_centroid(y=audio, sr=sr)[0]
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=audio, sr=sr)[0]
    spectral_rolloff = librosa.feature.spectral_rolloff(y=audio, sr=sr)[0]
    rmse = librosa.feature.rms(y=audio)[0]
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y=audio)[0]

    # Concatenate features into a single feature vector
    features = np.hstack(
        (chroma_stft.T, spectral_centroid.reshape(-1, 1), spectral_bandwidth.reshape(-1, 1),
         spectral_rolloff.reshape(-1, 1), rmse.reshape(-1, 1), zero_crossing_rate.reshape(-1, 1))
    )

    # Scale the features
    scaled_features = scaler.transform(features)

    # Classify the audio sample
    predicted_label = classifier.predict(scaled_features)

    return predicted_label[0]  # Return the predicted label as a string

# Use the classify_audio function to classify a single WAV file
wav_file_path = 'sound_403.wav'  # Replace with the path to your input WAV file

predicted_label = classify_audio(wav_file_path)

# Load the label encoders
ambulance_label_encoder = joblib.load('ambulance_label_encoder.pkl')
traffic_label_encoder = joblib.load('traffic_label_encoder.pkl')

# Map the predicted label back to the original class name
if predicted_label == ambulance_label_encoder.transform(['ambulance']):
    predicted_label = 'ambulance'
elif predicted_label == traffic_label_encoder.transform(['traffic']):
    predicted_label = 'traffic'

print("Predicted Label:", predicted_label)


ValueError: ignored

In [115]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import librosa
from scipy.io import wavfile
import joblib

# Load the merged dataset
general_data = pd.read_csv('general.csv')

# Separate features and labels
features = general_data.iloc[:, :-1]
labels = general_data.iloc[:, -1]

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, encoded_labels, test_size=0.2, random_state=42)

# Select the relevant features for scaling
selected_features = ['chroma_stft', 'spectral_centroid', 'spectral_bandwidth', 'spectral_rolloff', 'rmse', 'zero_crossing_rate']
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

# Scale the selected features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_selected)
X_test_scaled = scaler.transform(X_test_selected)

# Train the classifier
classifier = SVC()
classifier.fit(X_train_scaled, y_train)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

# Save the classifier model and label encoder
joblib.dump(classifier, 'classifier_model.pkl')
joblib.dump(label_encoder, 'label_encoder.pkl')

# Define a function to classify a single WAV file
def classify_audio(file_path):
    audio, sr = librosa.load(file_path, sr=None)  # Load the audio file

    # Extract features
    chroma_stft = librosa.feature.chroma_stft(y=audio, sr=sr)
    spectral_centroid = librosa.feature.spectral_centroid(y=audio, sr=sr)[0]
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=audio, sr=sr)[0]
    spectral_rolloff = librosa.feature.spectral_rolloff(y=audio, sr=sr)[0]
    rmse = librosa.feature.rms(y=audio)[0]
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y=audio)[0]

    # Concatenate features into a single feature vector
    features = np.hstack(
        (chroma_stft.T, spectral_centroid.reshape(-1, 1), spectral_bandwidth.reshape(-1, 1),
         spectral_rolloff.reshape(-1, 1), rmse.reshape(-1, 1), zero_crossing_rate.reshape(-1, 1))
    )

    # Select the relevant features for scaling
    selected_features = features[:, :6]

    # Scale the selected features
    scaled_features = scaler.transform(selected_features)

    # Classify the audio sample
    predicted_label = classifier.predict(scaled_features)

    # Inverse transform the predicted label to get the original class name
    predicted_class = label_encoder.inverse_transform(predicted_label)

    return predicted_class[0]  # Return the predicted label as a string

# Use the classify_audio function to classify a single WAV file
wav_file_path = 'sound_444.wav'  # Replace with the path to your input WAV file

predicted_label = classify_audio(wav_file_path)
print("Predicted Label:", predicted_label)


Accuracy: 0.9125
Precision: 0.9024390243902439
Recall: 0.925
F1 Score: 0.9135802469135802
Predicted Label: ambulance


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [116]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import librosa
from scipy.io import wavfile
import joblib

# Load the merged dataset
general_data = pd.read_csv('general.csv')

# Separate features and labels
features = general_data.iloc[:, :-1]
labels = general_data.iloc[:, -1]

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, encoded_labels, test_size=0.2, random_state=42)

# Define the selected feature columns
selected_features = ['chroma_stft', 'spectral_centroid', 'spectral_bandwidth', 'spectral_rolloff', 'rmse', 'zero_crossing_rate']

# Extract selected features for training set
X_train_selected = X_train[selected_features]

# Extract selected features for testing set
X_test_selected = X_test[selected_features]

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_selected)
X_test_scaled = scaler.transform(X_test_selected)

# Train the classifier
classifier = SVC()
classifier.fit(X_train_scaled, y_train)

# Save the classifier model and label encoder
joblib.dump(classifier, 'classifier_model.pkl')
joblib.dump(label_encoder, 'label_encoder.pkl')

# Define a function to classify a single WAV file
def classify_audio(file_path):
    audio, sr = librosa.load(file_path, sr=None)  # Load the audio file

    # Extract features
    chroma_stft = librosa.feature.chroma_stft(y=audio, sr=sr)
    spectral_centroid = librosa.feature.spectral_centroid(y=audio, sr=sr)[0]
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=audio, sr=sr)[0]
    spectral_rolloff = librosa.feature.spectral_rolloff(y=audio, sr=sr)[0]
    rmse = librosa.feature.rms(y=audio)[0]
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y=audio)[0]

    # Create a feature matrix from extracted features
    features = np.vstack([chroma_stft, spectral_centroid, spectral_bandwidth, spectral_rolloff, rmse, zero_crossing_rate]).T

    # Select the relevant features for scaling
    selected_features = features[:, :6]

    # Scale the selected features
    scaled_features = scaler.transform(selected_features)

    # Classify the audio sample
    predicted_label = classifier.predict(scaled_features)

    # Inverse transform the predicted label to get the original class name
    predicted_class = label_encoder.inverse_transform(predicted_label)

    return predicted_class[2]  # Return the predicted label as a string

# Use the classify_audio function to classify a single WAV file
wav_file_path = 'sound_444.wav'  # Replace with the path to your input WAV file

predicted_label = classify_audio(wav_file_path)
print("Predicted Label:", predicted_label)


Predicted Label: ambulance


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [107]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
import librosa
from scipy.io import wavfile
import joblib

# Load the merged dataset
general_data = pd.read_csv('general.csv')

# Separate features and labels
features = general_data.iloc[:, :6]
labels = general_data.iloc[:, -1]

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Select the relevant features for scaling
selected_features = ['chroma_stft', 'spectral_centroid', 'spectral_bandwidth', 'spectral_rolloff', 'rmse', 'zero_crossing_rate']
selected_features_data = features[selected_features]

# Scale the selected features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(selected_features_data)

# Train the classifier
classifier = SVC()
classifier.fit(scaled_features, encoded_labels)

# Save the classifier model and label encoder
joblib.dump(classifier, 'classifier_model.pkl')
joblib.dump(label_encoder, 'label_encoder.pkl')

# Define a function to classify a single WAV file
def classify_audio(file_path):
    audio, sr = librosa.load(file_path, sr=None)  # Load the audio file

    # Extract features
    chroma_stft = librosa.feature.chroma_stft(y=audio, sr=sr)
    spectral_centroid = librosa.feature.spectral_centroid(y=audio, sr=sr)[0]
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=audio, sr=sr)[0]
    spectral_rolloff = librosa.feature.spectral_rolloff(y=audio, sr=sr)[0]
    rmse = librosa.feature.rms(y=audio)[0]
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y=audio)[0]

    # Create a feature vector
    features = np.hstack(
        (chroma_stft.T, spectral_centroid.reshape(-1, 1), spectral_bandwidth.reshape(-1, 1),
         spectral_rolloff.reshape(-1, 1), rmse.reshape(-1, 1), zero_crossing_rate.reshape(-1, 1))
    )

    # Select the relevant features for scaling
    selected_features = features[:, :6]

    # Scale the selected features
    scaled_features = scaler.transform(selected_features)

    # Classify the audio sample
    predicted_label = classifier.predict(scaled_features)

    # Inverse transform the predicted label to get the original class name
    predicted_class = label_encoder.inverse_transform(predicted_label)

    return predicted_class[0]  # Return the predicted label as a string

# Use the classify_audio function to classify the WAV file
wav_file_path = 'sound_403.wav'  # Replace with the path to your input WAV file

predicted_label = classify_audio(wav_file_path)
print("Predicted Label:", predicted_label)


Predicted Label: ambulance


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [95]:
labels = general_data.iloc[:, -1]

In [96]:
labels

0      ambulance
1      ambulance
2      ambulance
3      ambulance
4      ambulance
         ...    
395      traffic
396      traffic
397      traffic
398      traffic
399      traffic
Name: label, Length: 400, dtype: object

In [99]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
import librosa

# Load the merged dataset
general_data = pd.read_csv('general.csv')

# Separate features and labels
features = general_data.iloc[:, :-1]
labels = general_data.iloc[:, -1]

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Scale the features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# Train the classifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(scaled_features, encoded_labels)

# Save the classifier model and label encoder
joblib.dump(classifier, 'classifier_model.pkl')
joblib.dump(label_encoder, 'label_encoder.pkl')

# Define a function to classify a single WAV file
def classify_audio(file_path):
    audio, sr = librosa.load(file_path, sr=None)  # Load the audio file

    # Extract features
    chroma_stft = librosa.feature.chroma_stft(y=audio, sr=sr)
    spectral_centroid = librosa.feature.spectral_centroid(y=audio, sr=sr)[0]
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=audio, sr=sr)[0]
    spectral_rolloff = librosa.feature.spectral_rolloff(y=audio, sr=sr)[0]
    rmse = librosa.feature.rms(y=audio)[0]
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y=audio)[0]

    # Create a feature vector
    features = np.array([chroma_stft[0][0], spectral_centroid[0], spectral_bandwidth[0],
                         spectral_rolloff[0], rmse[0], zero_crossing_rate[0]]).reshape(1, -1)

    # Scale the features
    scaled_features = scaler.transform(features)

    # Classify the audio sample
    predicted_label = classifier.predict(scaled_features)

    # Inverse transform the predicted label to get the original class name
    predicted_class = label_encoder.inverse_transform(predicted_label)

    return predicted_class[0]  # Return the predicted label as a string

# Use the classify_audio function to classify the WAV file
wav_file_path = 'sound_403.wav'  # Replace with the path to your input WAV file

predicted_label = classify_audio(wav_file_path)
print("Predicted Label:", predicted_label)


Predicted Label: ambulance


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [111]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
import librosa

# Load the dataset
data = pd.read_csv('general.csv')

# Select relevant features for training
features = data.iloc[:, :6]
labels = data.iloc[:, -1]

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, encoded_labels, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape the feature arrays for compatibility with CNN input
X_train_reshaped = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
X_test_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

# Define the CNN model
model = Sequential()
model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train_reshaped.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_reshaped, y_train, batch_size=32, epochs=10, verbose=1)

# Save the trained model and label encoder
model.save('classifier_model.h5')
joblib.dump(label_encoder, 'label_encoder.pkl')

# Define a function to classify a single WAV file
def classify_audio(file_path):
    audio, sr = librosa.load(file_path, sr=None)  # Load the audio file

    # Extract features
    chroma_stft = librosa.feature.chroma_stft(y=audio, sr=sr)[0]
    spectral_centroid = librosa.feature.spectral_centroid(y=audio, sr=sr)[0]
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=audio, sr=sr)[0]
    spectral_rolloff = librosa.feature.spectral_rolloff(y=audio, sr=sr)[0]
    rmse = librosa.feature.rms(y=audio)[0]
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y=audio)[0]

    # Create a feature vector
    features = np.concatenate([chroma_stft, spectral_centroid, spectral_bandwidth,
                         spectral_rolloff, rmse, zero_crossing_rate])

    # Scale the features
    scaled_features = scaler.transform([features[:6]])

    # Reshape the feature array for compatibility with CNN input
    reshaped_features = scaled_features.reshape(1, scaled_features.shape[1], 1)

    # Classify the audio sample
    predicted_prob = model.predict(reshaped_features)
    predicted_label = (predicted_prob > 0.5).astype(int)

    # Inverse transform the predicted label to get the original class name
    predicted_class = label_encoder.inverse_transform(predicted_label)[0]

    return predicted_class  # Return the predicted label as a string

# Use the classify_audio function to classify the WAV file
wav_file_path = 'sound_403.wav'  # Replace with the path to your input WAV file

predicted_label = classify_audio(wav_file_path)
print("Predicted Label:", predicted_label)


Epoch 1/10
10/10 [==============================] - 2s 5ms/step - loss: 0.6640 - accuracy: 0.5750
Epoch 2/10
10/10 [==============================] - 0s 7ms/step - loss: 0.5883 - accuracy: 0.8000
Epoch 3/10
10/10 [==============================] - 0s 7ms/step - loss: 0.5291 - accuracy: 0.9000
Epoch 4/10
10/10 [==============================] - 0s 5ms/step - loss: 0.4718 - accuracy: 0.9094
Epoch 5/10
10/10 [==============================] - 0s 9ms/step - loss: 0.4357 - accuracy: 0.9187
Epoch 6/10
10/10 [==============================] - 0s 7ms/step - loss: 0.3881 - accuracy: 0.9219
Epoch 7/10
10/10 [==============================] - 0s 8ms/step - loss: 0.3347 - accuracy: 0.9281
Epoch 8/10
10/10 [==============================] - 0s 8ms/step - loss: 0.3192 - accuracy: 0.9219
Epoch 9/10
10/10 [==============================] - 0s 8ms/step - loss: 0.2722 - accuracy: 0.9375
Epoch 10/10
10/10 [==============================] - 0s 7ms/step - loss: 0.2530 - accuracy: 0.9406


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 129ms/step
Predicted Label: ambulance


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:155: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [127]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import load_model
import librosa

# Load the dataset
data = pd.read_csv('general.csv')

# Select relevant features for training
features = data.iloc[:, :6]
labels = data.iloc[:, -1]

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Scale the features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# Load the saved model
model = load_model('classifier_model.h5')

# Define a function to preprocess the audio
def preprocess_audio(file_path):
    audio, sr = librosa.load(file_path, sr=None)  # Load the audio file

    # Extract features
    chroma_stft = np.mean(librosa.feature.chroma_stft(y=audio, sr=sr), axis=1)
    spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=audio, sr=sr))
    spectral_bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=audio, sr=sr))
    spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=audio, sr=sr))
    rmse = np.mean(librosa.feature.rms(y=audio))
    zero_crossing_rate = np.mean(librosa.feature.zero_crossing_rate(y=audio))

    # Create a feature vector
    features = np.array([chroma_stft, spectral_centroid, spectral_bandwidth,
                         spectral_rolloff, rmse, zero_crossing_rate])

    return features

# Define a function to classify a single WAV file
def classify_audio(file_path):
    # Preprocess the audio
    features = preprocess_audio(file_path)

    # Scale the features
    scaled_features = scaler.transform([features])

    # Classify the audio sample
    predicted_prob = model.predict(np.expand_dims(scaled_features, axis=2))
    predicted_label = (predicted_prob > 0.5).astype(int)

    # Inverse transform the predicted label to get the original class name
    predicted_class = label_encoder.inverse_transform(predicted_label)[0]

    return predicted_class  # Return the predicted label as a string

# Use the classify_audio function to classify the WAV file
wav_file_path = 'sound_1.wav'  # Replace with the path to your input WAV file

predicted_label = classify_audio(wav_file_path)
print("Predicted Label:", predicted_label)


<ipython-input-127-c39ee26535e1>:38: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  features = np.array([chroma_stft, spectral_centroid, spectral_bandwidth,
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


ValueError: ignored